# multi-threading com julia ༜

### ⋆ controle do número de threads
1. argumento de comando: `-t` ou `--threads`
2. variável de ambiente: `JULIA_NUM_THREADS`

obs: pode ser testado com `Threads.nthreads()`

### ⋆ funções e macros
  *Threads.threadid()*
    
  Returna id da thread em execução (thread mestre tem id=1)

  *Thread.@spawn t*

  Aloca a task t para uma thread disponível
  
  Pode ser alocada para mais de uma thread em caso de loop

  *fetch*
  
  Bloqueia a execução do programa até que a tarefa termine

In [ ]:
x = Threads.@spawn begin
    print("thread ", Threads.threadid(), " somou: " ) #apenas uma thread irá realizar a soma
    sum(1:1_000_000)
end

print(fetch(x))

In [ ]:
tasks = []
for i in 1:4
    push!(tasks, Threads.@spawn begin # cada thread irá realizar a soma de 1000 elementos por vez
            println(Threads.threadid())
            sum((i-1)*1_000+1:i*1_000)
        end)
end

total = sum(fetch.(tasks)) # o ponto é para aplicar a função fetch a cada elemento do vetor tasks
print(total)

  *Thread.@threads*

  Usada para paralelizar loops, dividindo automaticamente as iterações entre as threads disponíveis

  Não precisa de `fetch`, pois ela garante que todas as iterações serão executadas

In [ ]:
Threads.@threads for i in 1:4
    println("Thread $(Threads.threadid()) executando iteração $i")
end

In [ ]:
sum = 0
Threads.@threads for i in 1:10
    global sum += i
end

println(sum)

### ⋆ condições de corrida

Ocorrem em programas paralelos quando múltiplas threads ou processos tentam acessar ou modificar uma mesma variável ou recurso ao mesmo tempo. O resultado final pode depender da **ordem de execução das threads**, que não é garantida. Operações intermediárias podem ser interrompidas, resultando em atualizações incorretas ou valores inesperados.

In [ ]:
# exemplo
soma = 0

Threads.@threads for i in 1:10
    global soma += i  # Várias threads acessam/modificam a variável ao mesmo tempo
end

println(soma)

###  como evitar?
  1. Uso de variáveis atômicas

In [ ]:
using Base.Threads

soma = Atomic{Int}(0)

Threads.@threads for i in 1:10
    atomic_add!(soma, i)  # Atualização segura
end

println(soma[])  # Acessa o valor real da variável atômica

  2. Uso de *locks*

In [ ]:
using Base.Threads

my_lock = ReentrantLock()  # Cria a trava
soma = 0

Threads.@threads for i in 1:10
    @lock my_lock global soma += i
end

println(soma)